In [1]:
from datasets import flickr8k_parse
from keras import backend as K
from keras import Model
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Input, Dense, LSTM, add, Embedding, GRU, Dropout, Multiply, Dot, Lambda, BatchNormalization, \
    RepeatVector, concatenate
from keras.optimizers import RMSprop
from keras.backend.tensorflow_backend import set_session
from models import batch_generator, decoder

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import path_generation
import tensorflow as tf
import text_processing
import time

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2498995129509002572, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1462032793
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10982203293942572571
 physical_device_desc: "device: 0, name: GeForce 840M, pci bus id: 0000:07:00.0, compute capability: 5.0"]

# Decoder

### Captions encoding

Before building decoder, it is necessary to encode captions into one-hot vectors which further would be used in embedding layer.

### COCO dataset

In [3]:
# captions_path = 'D:/coco/annotations/'
# images_path = 'D:/coco/images/'

# # parse JSON file with captions to get paths to images with captions
# val_filenames_with_captions = coco_parse.get_image_filename_with_caption(captions_path, images_path, 
#                                                                      train=False)
# val_filenames_with_all_captions = coco_parse.get_image_with_all_captions(val_filenames_with_captions)

# train_filenames_with_captions = coco_parse.get_image_filename_with_caption(captions_path, images_path, 
#                                                                      train=True)
# train_filenames_with_all_captions = coco_parse.get_image_with_all_captions(train_filenames_with_captions)

# ### Extract captions
# train_captions = coco_parse.make_list_of_captions(train_filenames_with_all_captions)
# val_captions = coco_parse.make_list_of_captions(val_filenames_with_all_captions)

### Flickr8k dataset

In [4]:
images_path = 'D:/Flickr8k/images/'
annotations_path = 'D:/Flickr8k/annotations/'
captions_file = 'D:/Flickr8k/annotations/Flickr8k.token.txt'
train_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.trainImages.txt'
dev_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.devImages.txt'
test_txt_path = 'D:/Flickr8k/annotations/Flickr_8k.testImages.txt'

filenames_with_all_captions = flickr8k_parse.generate_filenames_with_all_captions(captions_file, images_path)

train_filenames_with_all_captions = flickr8k_parse.generate_set(train_txt_path, filenames_with_all_captions, images_path)
val_filenames_with_all_captions = flickr8k_parse.generate_set(dev_txt_path, filenames_with_all_captions, images_path)
test_filenames_with_all_captions = flickr8k_parse.generate_set(test_txt_path, filenames_with_all_captions, images_path)

train_captions = flickr8k_parse.make_list_of_captions(train_filenames_with_all_captions)
val_captions = flickr8k_parse.make_list_of_captions(val_filenames_with_all_captions)

In [5]:
### Preprocess captions
text_processing.preprocess_captions(val_captions)
text_processing.preprocess_captions(train_captions)

In [6]:
### Add markers of captions' starts and ends
text_processing.add_start_and_end_to_captions(train_captions)
text_processing.add_start_and_end_to_captions(val_captions)

In [7]:
### Create vocabulary from the training captions
train_vocab = text_processing.Vocabulary()
for caption_list in train_captions:
    for caption in caption_list:
        tmp_caption_list = caption.split()
        for word in tmp_caption_list:
            train_vocab.add_word(word)

In [8]:
if not os.path.exists('./vocabulary'):
    os.mkdir('./vocabulary')
train_vocab.save_vocabulary('word_to_id.pickle', 'id_to_word.pickle', 'word_counter.pickle')

In [9]:
train_captions_tokens = text_processing.tokenise_captions(train_captions, train_vocab)
val_captions_tokens = text_processing.tokenise_captions(val_captions, train_vocab)

In [10]:
train_captions_tokens[0]

[[1, 2, 3, 4, 5, 6, 7, 2, 8, 4, 9, 10, 11, 12],
 [1, 3, 4, 13, 14, 4, 15, 11, 12],
 [1, 16, 17, 18, 19, 20, 21, 10, 22, 23, 12],
 [1, 16, 17, 24, 25, 9, 10, 11, 12],
 [1, 16, 17, 6, 15, 2, 26, 27, 28, 29, 30, 12]]

In [11]:
train_captions[0]

['<sos> a black dog is running after a white dog in the snow <eos>',
 '<sos> black dog chasing brown dog through snow <eos>',
 '<sos> two dogs chase each other across the snowy ground <eos>',
 '<sos> two dogs play together in the snow <eos>',
 '<sos> two dogs running through a low lying body of water <eos>']

### Decoder NN

### GRU

In [12]:
dataset = 'flickr8k'
batch_size = 32
epochs = 30
steps_per_epoch = int(len(train_captions) / batch_size)
initial_state_size = 512
embedding_out_size = 512
number_of_layers = 2
batch_norm = True
dropout = True
gru = False
attn = True
attn_type = 'bahdanau'
max_len = 30
path_gen = path_generation.PathGenerator(gru, dataset, number_of_layers, batch_size, batch_norm, dropout, attn, attn_type)
path_checkpoint = path_gen.get_weights_path()
model_path = path_gen.get_model_path()
callbacks_path = path_gen.get_callbacks_path()

In [13]:
print(callbacks_path)

./model_files/callbacks/VGG16_LSTM_flickr8k_2l_32b_bn_dr_attn_bahdanau.csv


In [14]:
if attn:
    transfer_values = np.load('./cnn_features/vgg16_flickr8k_train_attn.npy')
    val_transfer_values = np.load('./cnn_features/vgg16_flickr8k_val_attn.npy')
else:
    transfer_values = np.load('./cnn_features/vgg16_flickr8k_train.npy')
    val_transfer_values = np.load('./cnn_features/vgg16_flickr8k_val.npy')

In [15]:
if attn:
    print(transfer_values.shape)
    transfer_values = transfer_values.reshape(6000, -1, 512)
    val_transfer_values = val_transfer_values.reshape(1000, -1, 512)
    print(transfer_values.shape)

(6000, 14, 14, 512)
(6000, 196, 512)


In [16]:
decoder_model = decoder.Decoder(initial_state_size,
                               embedding_out_size,
                               number_of_layers,
                               gru,
                               batch_norm,
                               dropout,
                               attn,
                               attn_type,
                               transfer_values,
                               train_vocab)
decoder_model = decoder_model.build_model()

Initial features shape (?, 196, 512)
Instructions for updating:
Colocations handled automatically by placer.
word-embedding (?, 30, 512)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Initial states
s initial (?, 512)
c initial (?, 512)
------------------------
LSTM iteration 0
------------------------
Attention
img features (?, 196, 512)
prev state (?, 512)
a_dense (?, 196, 512)
s_dense (?, 1, 512)
summary (?, 196, 512)
first_dense (?, 196, 512)
weights (?, 196, 1)
context (?, 1, 512)
------------------------
context (?, 1, 512)
current word vector (?, 1, 512)
lstm input: context-word concat (?, 1, 1024)
hidden state (?, 512)
------------------------
LSTM iteration 1
------------------------
Attention
img features (?, 196, 512)
prev state (?, 512)
a_dense (?, 196, 512)
s_dense (?, 1, 512)
summary (?, 196, 512)
first_dense (?, 196, 512)
weights (?, 196, 1)
context (?, 1, 512)
------------------------
context (?, 1, 512

In [17]:
if gru:
    generator = batch_generator.generate_batch(transfer_values, train_captions_tokens, number_of_words=train_vocab.number_of_words, batch_size=batch_size)
    val_generator = batch_generator.generate_batch(val_transfer_values, val_captions_tokens, number_of_words=train_vocab.number_of_words, batch_size=batch_size)
else:
    generator = batch_generator.generate_batch(transfer_values, train_captions_tokens, number_of_words=train_vocab.number_of_words, batch_size=batch_size, gru=False)
    val_generator = batch_generator.generate_batch(val_transfer_values, val_captions_tokens, number_of_words=train_vocab.number_of_words, batch_size=batch_size, gru=False)              

In [18]:
optimizer = RMSprop(lr=1e-3, decay=1e-8)

In [19]:
decoder_model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy')

In [20]:
decoder_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 196, 512)     0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           encoder_input[0][0]              
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 512)          262656      lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 512)          2048        dense_5[0][0]                    
__________________________________________________________________________________________________
lambda_3 (

__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 512)          2048        dense_6[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 512), (None, 3147776     concatenate_1[0][0]              
                                                                 batch_normalization_5[0][0]      
                                                                 batch_normalization_6[0][0]      
                                                                 concatenate_2[0][0]              
                                                                 lstm_1[0][0]                     
                                                                 lstm_1[0][2]                     
                                                                 concatenate_3[0][0]              
          

                                                                 encoder_input[0][0]              
__________________________________________________________________________________________________
multiply_5 (Multiply)           (None, 1, 512)       0           dot_5[0][0]                      
                                                                 dense_3[4][0]                    
__________________________________________________________________________________________________
lambda_17 (Lambda)              (None, 1, 512)       0           dropout_1[0][0]                  
__________________________________________________________________________________________________
concatenate_5 (Concatenate)     (None, 1, 1024)      0           multiply_5[0][0]                 
                                                                 lambda_17[0][0]                  
__________________________________________________________________________________________________
lambda_18 

__________________________________________________________________________________________________
lambda_41 (Lambda)              (None, 1, 512)       0           dropout_1[0][0]                  
__________________________________________________________________________________________________
concatenate_13 (Concatenate)    (None, 1, 1024)      0           multiply_13[0][0]                
                                                                 lambda_41[0][0]                  
__________________________________________________________________________________________________
lambda_42 (Lambda)              (None, 1, 512)       0           lstm_1[12][0]                    
__________________________________________________________________________________________________
add_14 (Add)                    (None, 196, 512)     0           dense_2[13][0]                   
                                                                 dense_1[13][0]                   
__________

                                                                 lambda_65[0][0]                  
__________________________________________________________________________________________________
lambda_66 (Lambda)              (None, 1, 512)       0           lstm_1[20][0]                    
__________________________________________________________________________________________________
add_22 (Add)                    (None, 196, 512)     0           dense_2[21][0]                   
                                                                 dense_1[21][0]                   
__________________________________________________________________________________________________
lambda_67 (Lambda)              (None, 196, 512)     0           add_22[0][0]                     
__________________________________________________________________________________________________
weights_21 (Lambda)             (None, 196, 1)       0           dense_4[21][0]                   
__________

add_30 (Add)                    (None, 196, 512)     0           dense_2[29][0]                   
                                                                 dense_1[29][0]                   
__________________________________________________________________________________________________
lambda_91 (Lambda)              (None, 196, 512)     0           add_30[0][0]                     
__________________________________________________________________________________________________
weights_29 (Lambda)             (None, 196, 1)       0           dense_4[29][0]                   
__________________________________________________________________________________________________
dot_30 (Dot)                    (None, 1, 512)       0           weights_29[0][0]                 
                                                                 encoder_input[0][0]              
__________________________________________________________________________________________________
multiply_3

In [21]:
model_json = decoder_model.to_json()
try:
    os.mkdir('./models')
except:
    print('The folder already exists')
with open(model_path, "w") as json_file:
    json.dump(json.loads(model_json), json_file, indent=4)

D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'batch_normalization_5/cond/Merge:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'batch_normalization_6/cond/Merge:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword argument

D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_19/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1_19/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_20/TensorArrayReadV3:0' shape=(?, 512) dtype=float32>, <tf.Tensor 'lstm_1_20/while/Exit_4:0' shape=(?, 512) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '
D:\Anaconda\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_1 was passed non-serializable keyword arguments: {'init

The folder already exists


### Checkpoints

During the training process, it is a good idea to save the weights periodically.

In [22]:
try:
    os.mkdir('./weights/')
except:
    print('The folder already exists')

checkpoints = ModelCheckpoint(path_checkpoint, verbose=1, save_weights_only=True, save_best_only=False)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=2, verbose=1, min_lr=0.0000001)

The folder already exists


In [23]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))
start = time.time()
callbacks = decoder_model.fit_generator(generator=generator,
                            steps_per_epoch=steps_per_epoch,
                            epochs=epochs,
                            callbacks=[checkpoints, reduce_lr],
                            validation_data=val_generator,
                            validation_steps=5)
time_train = time.time() - start

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/30
187/187 [==============================] - ETA: 3:28:46 - loss: 8.96 - ETA: 1:46:19 - loss: 7.95 - ETA: 1:12:07 - loss: 7.12 - ETA: 55:00 - loss: 6.5388 - ETA: 44:44 - loss: 6.03 - ETA: 37:53 - loss: 5.79 - ETA: 32:59 - loss: 5.48 - ETA: 29:17 - loss: 5.19 - ETA: 26:25 - loss: 4.96 - ETA: 24:07 - loss: 4.74 - ETA: 22:14 - loss: 4.55 - ETA: 20:39 - loss: 4.39 - ETA: 19:20 - loss: 4.25 - ETA: 18:12 - loss: 4.11 - ETA: 17:13 - loss: 3.98 - ETA: 16:21 - loss: 3.88 - ETA: 15:35 - loss: 3.77 - ETA: 14:54 - loss: 3.68 - ETA: 14:17 - loss: 3.60 - ETA: 13:44 - loss: 3.53 - ETA: 13:14 - loss: 3.47 - ETA: 12:46 - loss: 3.41 - ETA: 12:20 - loss: 3.35 - ETA: 11:57 - loss: 3.31 - ETA: 11:35 - loss: 3.25 - ETA: 11:15 - loss: 3.20 - ETA: 10:56 - loss: 3.17 - ETA: 10:38 - loss: 3.13 - ETA: 10:22 - loss: 3.09 - ETA: 10:06 - loss: 3.06 - ETA: 9:51 - loss: 3.0299 - ETA: 

187/187 [==============================] - ETA: 5:02 - loss: 1.334 - ETA: 4:57 - loss: 1.410 - ETA: 4:55 - loss: 1.431 - ETA: 4:54 - loss: 1.392 - ETA: 4:52 - loss: 1.357 - ETA: 4:50 - loss: 1.347 - ETA: 4:48 - loss: 1.337 - ETA: 4:47 - loss: 1.305 - ETA: 4:45 - loss: 1.324 - ETA: 4:43 - loss: 1.332 - ETA: 4:42 - loss: 1.330 - ETA: 4:40 - loss: 1.335 - ETA: 4:39 - loss: 1.336 - ETA: 4:37 - loss: 1.354 - ETA: 4:36 - loss: 1.356 - ETA: 4:34 - loss: 1.355 - ETA: 4:32 - loss: 1.355 - ETA: 4:31 - loss: 1.362 - ETA: 4:29 - loss: 1.363 - ETA: 4:27 - loss: 1.368 - ETA: 4:26 - loss: 1.371 - ETA: 4:24 - loss: 1.370 - ETA: 4:22 - loss: 1.370 - ETA: 4:21 - loss: 1.367 - ETA: 4:19 - loss: 1.366 - ETA: 4:17 - loss: 1.367 - ETA: 4:16 - loss: 1.370 - ETA: 4:14 - loss: 1.370 - ETA: 4:13 - loss: 1.374 - ETA: 4:11 - loss: 1.371 - ETA: 4:09 - loss: 1.372 - ETA: 4:08 - loss: 1.370 - ETA: 4:06 - loss: 1.371 - ETA: 4:05 - loss: 1.372 - ETA: 4:03 - loss: 1.372 - ETA: 4:01 - loss: 1.375 - ETA: 4:00 - loss: 1.3

187/187 [==============================] - ETA: 4:56 - loss: 1.400 - ETA: 4:54 - loss: 1.477 - ETA: 4:54 - loss: 1.458 - ETA: 4:52 - loss: 1.410 - ETA: 4:51 - loss: 1.379 - ETA: 4:49 - loss: 1.340 - ETA: 4:47 - loss: 1.327 - ETA: 4:46 - loss: 1.362 - ETA: 4:44 - loss: 1.361 - ETA: 4:42 - loss: 1.381 - ETA: 4:41 - loss: 1.385 - ETA: 4:39 - loss: 1.369 - ETA: 4:37 - loss: 1.365 - ETA: 4:36 - loss: 1.363 - ETA: 4:34 - loss: 1.353 - ETA: 4:33 - loss: 1.356 - ETA: 4:31 - loss: 1.352 - ETA: 4:29 - loss: 1.346 - ETA: 4:28 - loss: 1.340 - ETA: 4:26 - loss: 1.335 - ETA: 4:25 - loss: 1.329 - ETA: 4:23 - loss: 1.322 - ETA: 4:21 - loss: 1.315 - ETA: 4:20 - loss: 1.325 - ETA: 4:18 - loss: 1.320 - ETA: 4:17 - loss: 1.321 - ETA: 4:15 - loss: 1.321 - ETA: 4:14 - loss: 1.321 - ETA: 4:12 - loss: 1.314 - ETA: 4:10 - loss: 1.317 - ETA: 4:09 - loss: 1.315 - ETA: 4:07 - loss: 1.310 - ETA: 4:06 - loss: 1.308 - ETA: 4:04 - loss: 1.309 - ETA: 4:02 - loss: 1.313 - ETA: 4:01 - loss: 1.310 - ETA: 3:59 - loss: 1.3

187/187 [==============================] - ETA: 4:56 - loss: 1.308 - ETA: 4:55 - loss: 1.215 - ETA: 4:53 - loss: 1.187 - ETA: 4:52 - loss: 1.269 - ETA: 4:50 - loss: 1.296 - ETA: 4:48 - loss: 1.282 - ETA: 4:46 - loss: 1.248 - ETA: 4:45 - loss: 1.235 - ETA: 4:43 - loss: 1.231 - ETA: 4:42 - loss: 1.248 - ETA: 4:40 - loss: 1.247 - ETA: 4:39 - loss: 1.244 - ETA: 4:37 - loss: 1.236 - ETA: 4:36 - loss: 1.239 - ETA: 4:34 - loss: 1.235 - ETA: 4:32 - loss: 1.229 - ETA: 4:31 - loss: 1.239 - ETA: 4:29 - loss: 1.241 - ETA: 4:28 - loss: 1.237 - ETA: 4:26 - loss: 1.233 - ETA: 4:25 - loss: 1.235 - ETA: 4:23 - loss: 1.227 - ETA: 4:21 - loss: 1.223 - ETA: 4:20 - loss: 1.226 - ETA: 4:18 - loss: 1.233 - ETA: 4:17 - loss: 1.234 - ETA: 4:15 - loss: 1.234 - ETA: 4:13 - loss: 1.232 - ETA: 4:12 - loss: 1.240 - ETA: 4:10 - loss: 1.239 - ETA: 4:09 - loss: 1.240 - ETA: 4:07 - loss: 1.238 - ETA: 4:05 - loss: 1.234 - ETA: 4:04 - loss: 1.236 - ETA: 4:02 - loss: 1.236 - ETA: 4:01 - loss: 1.238 - ETA: 3:59 - loss: 1.2

187/187 [==============================] - ETA: 4:56 - loss: 1.355 - ETA: 4:54 - loss: 1.250 - ETA: 4:53 - loss: 1.269 - ETA: 4:51 - loss: 1.233 - ETA: 4:49 - loss: 1.219 - ETA: 4:48 - loss: 1.211 - ETA: 4:46 - loss: 1.207 - ETA: 4:45 - loss: 1.191 - ETA: 4:43 - loss: 1.191 - ETA: 4:42 - loss: 1.191 - ETA: 4:40 - loss: 1.196 - ETA: 4:39 - loss: 1.193 - ETA: 4:37 - loss: 1.182 - ETA: 4:36 - loss: 1.185 - ETA: 4:34 - loss: 1.195 - ETA: 4:32 - loss: 1.208 - ETA: 4:31 - loss: 1.202 - ETA: 4:29 - loss: 1.195 - ETA: 4:28 - loss: 1.197 - ETA: 4:26 - loss: 1.197 - ETA: 4:24 - loss: 1.194 - ETA: 4:23 - loss: 1.185 - ETA: 4:21 - loss: 1.186 - ETA: 4:20 - loss: 1.184 - ETA: 4:18 - loss: 1.194 - ETA: 4:16 - loss: 1.194 - ETA: 4:15 - loss: 1.196 - ETA: 4:13 - loss: 1.198 - ETA: 4:12 - loss: 1.197 - ETA: 4:10 - loss: 1.194 - ETA: 4:08 - loss: 1.199 - ETA: 4:07 - loss: 1.198 - ETA: 4:05 - loss: 1.198 - ETA: 4:04 - loss: 1.192 - ETA: 4:02 - loss: 1.193 - ETA: 4:00 - loss: 1.188 - ETA: 3:59 - loss: 1.1

...

187/187 [==============================] - ETA: 5:08 - loss: 0.988 - ETA: 5:06 - loss: 1.068 - ETA: 5:04 - loss: 1.000 - ETA: 5:02 - loss: 0.992 - ETA: 5:00 - loss: 0.996 - ETA: 4:58 - loss: 0.985 - ETA: 4:56 - loss: 0.989 - ETA: 4:54 - loss: 0.984 - ETA: 4:53 - loss: 0.995 - ETA: 4:51 - loss: 0.976 - ETA: 4:50 - loss: 0.976 - ETA: 4:48 - loss: 0.993 - ETA: 4:47 - loss: 0.995 - ETA: 4:45 - loss: 1.001 - ETA: 4:44 - loss: 1.000 - ETA: 4:42 - loss: 0.996 - ETA: 4:40 - loss: 0.987 - ETA: 4:39 - loss: 0.993 - ETA: 4:37 - loss: 1.000 - ETA: 4:35 - loss: 1.001 - ETA: 4:34 - loss: 1.002 - ETA: 4:32 - loss: 1.005 - ETA: 4:31 - loss: 1.007 - ETA: 4:29 - loss: 1.003 - ETA: 4:27 - loss: 1.005 - ETA: 4:26 - loss: 1.006 - ETA: 4:24 - loss: 1.011 - ETA: 4:22 - loss: 1.012 - ETA: 4:21 - loss: 1.013 - ETA: 4:19 - loss: 1.012 - ETA: 4:17 - loss: 1.011 - ETA: 4:16 - loss: 1.009 - ETA: 4:14 - loss: 1.010 - ETA: 4:12 - loss: 1.007 - ETA: 4:11 - loss: 1.008 - ETA: 4:09 - loss: 1.007 - ETA: 4:07 - loss: 1.0

187/187 [==============================] - ETA: 5:05 - loss: 1.002 - ETA: 5:05 - loss: 1.012 - ETA: 5:05 - loss: 1.039 - ETA: 5:03 - loss: 1.059 - ETA: 5:01 - loss: 1.027 - ETA: 4:59 - loss: 1.058 - ETA: 4:58 - loss: 1.031 - ETA: 4:57 - loss: 1.040 - ETA: 4:56 - loss: 1.047 - ETA: 4:55 - loss: 1.024 - ETA: 4:53 - loss: 1.015 - ETA: 4:51 - loss: 1.018 - ETA: 4:49 - loss: 1.018 - ETA: 4:47 - loss: 1.015 - ETA: 4:46 - loss: 1.011 - ETA: 4:44 - loss: 1.012 - ETA: 4:42 - loss: 1.012 - ETA: 4:40 - loss: 1.004 - ETA: 4:39 - loss: 1.004 - ETA: 4:37 - loss: 1.001 - ETA: 4:36 - loss: 1.005 - ETA: 4:34 - loss: 0.999 - ETA: 4:32 - loss: 0.996 - ETA: 4:30 - loss: 0.993 - ETA: 4:29 - loss: 0.994 - ETA: 4:27 - loss: 0.998 - ETA: 4:25 - loss: 0.997 - ETA: 4:24 - loss: 0.997 - ETA: 4:22 - loss: 0.997 - ETA: 4:20 - loss: 1.001 - ETA: 4:19 - loss: 1.005 - ETA: 4:17 - loss: 1.003 - ETA: 4:15 - loss: 1.004 - ETA: 4:14 - loss: 1.007 - ETA: 4:12 - loss: 1.010 - ETA: 4:11 - loss: 1.009 - ETA: 4:09 - loss: 1.0

187/187 [==============================] - ETA: 5:04 - loss: 1.028 - ETA: 5:04 - loss: 0.981 - ETA: 5:02 - loss: 1.007 - ETA: 5:01 - loss: 0.964 - ETA: 4:59 - loss: 0.961 - ETA: 4:58 - loss: 0.989 - ETA: 4:56 - loss: 0.983 - ETA: 4:55 - loss: 0.984 - ETA: 4:53 - loss: 0.991 - ETA: 4:51 - loss: 0.989 - ETA: 4:50 - loss: 0.984 - ETA: 4:48 - loss: 0.987 - ETA: 4:47 - loss: 0.989 - ETA: 4:45 - loss: 0.989 - ETA: 4:43 - loss: 0.993 - ETA: 4:42 - loss: 0.990 - ETA: 4:40 - loss: 0.985 - ETA: 4:38 - loss: 0.993 - ETA: 4:37 - loss: 0.995 - ETA: 4:35 - loss: 0.990 - ETA: 4:33 - loss: 0.984 - ETA: 4:32 - loss: 0.980 - ETA: 4:30 - loss: 0.978 - ETA: 4:29 - loss: 0.977 - ETA: 4:27 - loss: 0.978 - ETA: 4:25 - loss: 0.979 - ETA: 4:24 - loss: 0.981 - ETA: 4:22 - loss: 0.976 - ETA: 4:20 - loss: 0.976 - ETA: 4:19 - loss: 0.977 - ETA: 4:17 - loss: 0.984 - ETA: 4:15 - loss: 0.985 - ETA: 4:14 - loss: 0.984 - ETA: 4:12 - loss: 0.986 - ETA: 4:11 - loss: 0.988 - ETA: 4:09 - loss: 0.988 - ETA: 4:07 - loss: 0.9

187/187 [==============================] - ETA: 5:04 - loss: 0.869 - ETA: 5:04 - loss: 0.891 - ETA: 5:04 - loss: 0.976 - ETA: 5:03 - loss: 0.995 - ETA: 5:01 - loss: 1.000 - ETA: 5:00 - loss: 1.003 - ETA: 4:59 - loss: 0.995 - ETA: 4:57 - loss: 1.013 - ETA: 4:55 - loss: 1.007 - ETA: 4:53 - loss: 0.990 - ETA: 4:52 - loss: 0.993 - ETA: 4:50 - loss: 0.989 - ETA: 4:49 - loss: 0.989 - ETA: 4:47 - loss: 0.979 - ETA: 4:45 - loss: 0.999 - ETA: 4:43 - loss: 0.997 - ETA: 4:42 - loss: 0.993 - ETA: 4:40 - loss: 0.999 - ETA: 4:39 - loss: 0.997 - ETA: 4:37 - loss: 0.991 - ETA: 4:35 - loss: 0.982 - ETA: 4:34 - loss: 0.982 - ETA: 4:32 - loss: 0.988 - ETA: 4:30 - loss: 0.988 - ETA: 4:29 - loss: 0.986 - ETA: 4:27 - loss: 0.986 - ETA: 4:25 - loss: 0.986 - ETA: 4:24 - loss: 0.987 - ETA: 4:22 - loss: 0.988 - ETA: 4:20 - loss: 0.993 - ETA: 4:18 - loss: 0.989 - ETA: 4:17 - loss: 0.994 - ETA: 4:15 - loss: 0.995 - ETA: 4:13 - loss: 0.989 - ETA: 4:12 - loss: 0.992 - ETA: 4:10 - loss: 0.990 - ETA: 4:08 - loss: 0.9

187/187 [==============================] - ETA: 5:04 - loss: 1.003 - ETA: 5:03 - loss: 1.095 - ETA: 5:02 - loss: 1.039 - ETA: 5:01 - loss: 1.045 - ETA: 4:59 - loss: 1.061 - ETA: 4:57 - loss: 1.035 - ETA: 4:56 - loss: 1.043 - ETA: 4:54 - loss: 1.046 - ETA: 4:53 - loss: 1.038 - ETA: 4:51 - loss: 1.042 - ETA: 4:50 - loss: 1.034 - ETA: 4:48 - loss: 1.025 - ETA: 4:46 - loss: 1.029 - ETA: 4:45 - loss: 1.024 - ETA: 4:43 - loss: 1.031 - ETA: 4:42 - loss: 1.039 - ETA: 4:40 - loss: 1.042 - ETA: 4:38 - loss: 1.050 - ETA: 4:37 - loss: 1.047 - ETA: 4:35 - loss: 1.040 - ETA: 4:34 - loss: 1.050 - ETA: 4:32 - loss: 1.043 - ETA: 4:30 - loss: 1.034 - ETA: 4:29 - loss: 1.034 - ETA: 4:27 - loss: 1.034 - ETA: 4:25 - loss: 1.025 - ETA: 4:24 - loss: 1.024 - ETA: 4:22 - loss: 1.021 - ETA: 4:20 - loss: 1.025 - ETA: 4:19 - loss: 1.028 - ETA: 4:17 - loss: 1.030 - ETA: 4:15 - loss: 1.032 - ETA: 4:14 - loss: 1.029 - ETA: 4:12 - loss: 1.027 - ETA: 4:10 - loss: 1.029 - ETA: 4:09 - loss: 1.031 - ETA: 4:07 - loss: 1.0

In [24]:
print("Time for training: {} seconds".format(time_train))

Time for training: 9415.719113588333 seconds


In [25]:
if not os.path.exists('./callbacks'):
    os.mkdir('./callbacks')   
columns = callbacks.history.keys()

In [26]:
callback_df = pd.DataFrame(callbacks.history)
callback_df.to_csv(callbacks_path, index=None)